# Building a PCD/Stress Gene List for Waddington-OT Analysis
## *Brachypodium distachyon* Single-Cell RNA-seq

---

### Purpose

This notebook constructs a **Programmed Cell Death (PCD) and stress-related gene list** for use in Waddington-OT (WOT) trajectory analysis, starting from Gene Ontology (GO) annotations.

The gene list will serve as input to WOT's death-score calculation, which models cellular fate transitions by capturing gradients of stress and programmed cell death.

---

### Why PCD Instead of "Apoptosis"?

**Apoptosis** is a specific form of programmed cell death in animals. However, **plants lack the canonical apoptotic machinery** (caspases, Bcl-2 family proteins, etc.).

Instead, plant cells undergo:
- **Programmed Cell Death (PCD)**: Genetically controlled cell death (e.g., developmental processes, hypersensitive response)
- **Stress-induced cell death**: Triggered by oxidative stress, ROS, abiotic stress

Using GO terms specific to **plant Biological Processes** ensures biological relevance without relying on cross-species BLAST or projections from animal models.

---

### Methodological Rationale

1. **Native gene IDs**: *Brachypodium distachyon* Bd21-3 annotations (no BLAST required)
2. **GO-based selection**: Explicit GO terms for PCD, cell death regulation, oxidative stress, and ROS
3. **Deterministic ID mapping**: Protein-level annotations mapped to Seurat gene IDs (.v1.2 suffix)
4. **Biological Process only**: Excludes Molecular Function and Cellular Component to focus on processes
5. **Reproducibility**: All GO terms explicitly listed, script versioned

---

### Workflow Overview

```
GO annotations (eggNOG / HuaCM)
          │
          ▼
Protein-level GO terms
(BdiBd21-3.xGxxxxxxx.1.p)
          │
          ▼
Deterministic ID mapping
(BdiBd21-3.xGxxxxxxx.v1.2)
          │
          ▼
GO filtering (BP only)
PCD + Stress + ROS
          │
          ▼
PCD / death-like gene set
          │
          ▼
WOT death score
```

---

**Note**: This notebook generates ONLY the gene list. WOT trajectory analysis is performed separately.

## 1. Setup and Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import subprocess
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options for better readability
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✓ Libraries imported successfully")

## 2. Define Paths and Parameters

In [ ]:
# Input file: GO annotations from eggNOG/HuaCM
GO_ANNOTATION_PATH = Path("/Users/clpichot/Documents/BIO-INFO/GENOMES/GOannotation.tsv")

# Working directory and script
SCRIPT_PATH = Path("create_list_from_GAF.py")

# Output files
OUTPUT_DIR = Path(".")
GENE_LIST_RAW = OUTPUT_DIR / "PCD_WOT_genes.txt"
GENE_LIST_FINAL = OUTPUT_DIR / "PCD_WOT_genes.present_in_seurat.txt"

# Gene ID suffix for Seurat object
SEURAT_SUFFIX = ".v1.2"

print(f"GO annotation file: {GO_ANNOTATION_PATH}")
print(f"Extraction script: {SCRIPT_PATH}")
print(f"Output directory: {OUTPUT_DIR.resolve()}")

## 3. Define Target GO Terms

We use **10 GO Biological Process** terms covering:
- **Programmed Cell Death (PCD)**: Core death processes and their regulation
- **Oxidative Stress & ROS**: Reactive oxygen species metabolism and redox homeostasis
- **General Stress**: Cellular and abiotic stress responses

These terms are plant-appropriate and avoid animal-specific apoptosis pathways.

In [ ]:
# GO Biological Process terms for PCD and stress
GO_TERMS = {
    # === PCD / Cell Death ===
    "GO:0012501": "programmed cell death",
    "GO:0008219": "cell death",
    "GO:0043067": "regulation of programmed cell death",
    "GO:0010941": "regulation of cell death",
    
    # === Stress / ROS ===
    "GO:0006979": "response to oxidative stress",
    "GO:0072593": "reactive oxygen species metabolic process",
    "GO:0045454": "cell redox homeostasis",
    "GO:0033554": "cellular response to stress",
    "GO:0006950": "response to stress",
    "GO:0009628": "response to abiotic stimulus",
}

# Display GO terms
print(f"Total GO terms selected: {len(GO_TERMS)}\n")
print("GO ID       | Description")
print("-" * 70)
for go_id, description in GO_TERMS.items():
    print(f"{go_id} | {description}")

## 4. Load GO Annotation File

In [ ]:
# Load GO annotations
go_df = pd.read_csv(GO_ANNOTATION_PATH, sep="\t")

print(f"✓ Loaded {len(go_df):,} GO annotations")
print(f"\nColumns: {list(go_df.columns)}")
print(f"\nData shape: {go_df.shape}")
print(f"\nFirst 10 rows:")
go_df.head(10)

### Annotation Statistics

In [ ]:
# Basic statistics
print("GO Annotation Overview:")
print(f"  Total annotations: {len(go_df):,}")
print(f"  Unique genes: {go_df['gene'].nunique():,}")
print(f"  Unique GO terms: {go_df['GO'].nunique():,}")
print(f"\nGO level distribution:")
print(go_df['level'].value_counts())

# Check for target GO terms in the dataset
target_go_present = go_df[go_df['GO'].isin(GO_TERMS.keys())]
print(f"\n✓ Found {len(target_go_present):,} annotations matching our {len(GO_TERMS)} target GO terms")
print(f"  Unique genes with target GO terms: {target_go_present['gene'].nunique():,}")

## 5. ID Normalization: Protein → Gene Mapping

**Gene ID Format in GO Annotations (protein-level):**
```
BdiBd21-3.3G0362100.1.p
```

**Gene ID Format in Seurat Object:**
```
BdiBd21-3.3G0362100.v1.2
```

**Mapping Rule (deterministic):**
1. Extract core ID: `BdiBd21-3.xG0xxxxxxx0`
2. Append Seurat suffix: `.v1.2`

This mapping is **reversible** and ensures 1:1 correspondence between annotation and expression data.

In [ ]:
# Apply ID mapping to verify the conversion
# Pattern: Extract core ID (BdiBd21-3.xG0xxxxxxx0)
core_pattern = re.compile(r"^(BdiBd21-3\.\dG\d{7}0)")

# Test on a few examples
test_genes = go_df['gene'].head(5).tolist()
print("ID Mapping Examples:")
print(f"{'Protein ID (annotation)':<30} → {'Gene ID (Seurat)':<30}")
print("-" * 70)

for protein_id in test_genes:
    match = core_pattern.search(str(protein_id))
    if match:
        core_id = match.group(1)
        seurat_id = core_id + SEURAT_SUFFIX
        print(f"{protein_id:<30} → {seurat_id:<30}")
    else:
        print(f"{protein_id:<30} → [NO MATCH]")

# Apply mapping to full dataset
go_df['core'] = go_df['gene'].astype(str).str.extract(core_pattern, expand=False)
go_df_mapped = go_df.dropna(subset=['core']).copy()
go_df_mapped['seurat_gene'] = go_df_mapped['core'] + SEURAT_SUFFIX

print(f"\n✓ Successfully mapped {len(go_df_mapped):,} / {len(go_df):,} annotations")
print(f"  Unmapped: {len(go_df) - len(go_df_mapped):,}")

## 6. Extract PCD/Stress Genes

Now we filter for genes annotated with our target GO terms (Biological Process only) and extract unique Seurat gene IDs.

In [ ]:
# Filter for target GO terms and Biological Process only
pcd_annotations = go_df_mapped[
    (go_df_mapped['GO'].isin(GO_TERMS.keys())) & 
    (go_df_mapped['level'] == 'BP')
].copy()

print(f"PCD/Stress Annotations (BP only):")
print(f"  Total annotations: {len(pcd_annotations):,}")
print(f"  Unique genes: {pcd_annotations['seurat_gene'].nunique():,}")

# Extract unique gene list
pcd_genes = sorted(set(pcd_annotations['seurat_gene'].astype(str)))
print(f"\n✓ Extracted {len(pcd_genes):,} unique PCD/stress genes")

# Show first 10 genes
print(f"\nFirst 10 genes:")
for i, gene in enumerate(pcd_genes[:10], 1):
    print(f"  {i}. {gene}")

## 7. Quality Control: GO Term Distribution

Let's examine how genes are distributed across the different GO categories.

In [ ]:
# Count genes per GO term
go_counts = pcd_annotations.groupby('GO')['seurat_gene'].nunique().sort_values(ascending=False)

print("Gene Counts by GO Term:")
print("=" * 80)
for go_id, count in go_counts.items():
    description = GO_TERMS.get(go_id, "Unknown")
    print(f"{go_id} | {count:4d} genes | {description}")

# Create a summary dataframe
go_summary = pd.DataFrame({
    'GO_ID': go_counts.index,
    'Gene_Count': go_counts.values,
    'Description': [GO_TERMS.get(go_id, "Unknown") for go_id in go_counts.index]
})

go_summary

### Visualize GO Term Distribution

In [ ]:
# Create horizontal bar plot
fig, ax = plt.subplots(figsize=(12, 6))

# Prepare data for plotting (truncate long descriptions)
plot_data = go_summary.copy()
plot_data['Short_Desc'] = plot_data['Description'].str[:40]
plot_data = plot_data.sort_values('Gene_Count', ascending=True)

# Plot
bars = ax.barh(range(len(plot_data)), plot_data['Gene_Count'], color='steelblue')
ax.set_yticks(range(len(plot_data)))
ax.set_yticklabels([f"{row['GO_ID']}\n{row['Short_Desc']}" for _, row in plot_data.iterrows()], fontsize=9)
ax.set_xlabel('Number of Unique Genes', fontsize=11)
ax.set_title('PCD/Stress Gene Distribution Across GO Terms', fontsize=13, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, (idx, row) in enumerate(plot_data.iterrows()):
    ax.text(row['Gene_Count'] + 5, i, str(row['Gene_Count']), 
            va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n✓ Total unique genes across all GO terms: {len(pcd_genes):,}")

## 8. Check for Overlaps Between GO Categories

Some genes may be annotated with multiple GO terms. Let's quantify this.

In [ ]:
# Count how many GO terms each gene is annotated with
go_per_gene = pcd_annotations.groupby('seurat_gene')['GO'].nunique().sort_values(ascending=False)

print("Gene Multi-Annotation Statistics:")
print(f"  Total unique genes: {len(go_per_gene):,}")
print(f"  Genes with 1 GO term: {(go_per_gene == 1).sum():,}")
print(f"  Genes with 2+ GO terms: {(go_per_gene > 1).sum():,}")
print(f"  Max GO terms per gene: {go_per_gene.max()}")
print(f"\nDistribution of GO terms per gene:")
print(go_per_gene.value_counts().sort_index())

# Show top multi-annotated genes
print(f"\nTop 10 genes with most GO term annotations:")
for i, (gene, count) in enumerate(go_per_gene.head(10).items(), 1):
    terms = pcd_annotations[pcd_annotations['seurat_gene'] == gene]['GO'].tolist()
    print(f"  {i}. {gene}: {count} terms")
    print(f"     GO terms: {', '.join(terms)}")

## 9. Write Initial Gene List

Save the extracted genes to a file for reproducibility and potential manual inspection.

In [ ]:
# Write gene list to file (one gene per line)
GENE_LIST_RAW.write_text("\n".join(pcd_genes) + "\n")

print(f"✓ Wrote {len(pcd_genes):,} genes to: {GENE_LIST_RAW}")
print(f"\nFile preview (first 10 lines):")
with open(GENE_LIST_RAW, 'r') as f:
    for i, line in enumerate(f, 1):
        if i <= 10:
            print(f"  {i}. {line.strip()}")
        else:
            break

## 10. Filter for Genes Present in Seurat Object

**Important**: The final gene list should only include genes that are actually present in your Seurat single-cell object.

Since we don't have the Seurat object loaded here, we provide a template for filtering. You can either:

1. **Load your Seurat object** in R/Python and extract gene names
2. **Manually filter** this list against your Seurat features

For now, we'll create a **placeholder final file** that you should replace with the filtered version after checking against your Seurat object.

In [ ]:
# OPTION 1: If you have a file with Seurat gene names, uncomment and use:
# seurat_genes_file = Path("path/to/seurat_genes.txt")
# seurat_genes = set(pd.read_csv(seurat_genes_file, header=None)[0])
# 
# # Filter PCD genes for those present in Seurat
# pcd_genes_in_seurat = [gene for gene in pcd_genes if gene in seurat_genes]
# print(f"Genes in Seurat object: {len(pcd_genes_in_seurat):,} / {len(pcd_genes):,}")

# OPTION 2: For now, create the output file with all genes
# You should replace this with the filtered version
GENE_LIST_FINAL.write_text("\n".join(pcd_genes) + "\n")

print(f"✓ Created: {GENE_LIST_FINAL}")
print(f"\n⚠️  WARNING: This file contains ALL {len(pcd_genes):,} genes.")
print(f"   You should filter this list for genes present in your Seurat object.")
print(f"\nNext steps:")
print(f"  1. Load your Seurat object")
print(f"  2. Extract feature names (rownames(seurat))")
print(f"  3. Intersect with {GENE_LIST_RAW}")
print(f"  4. Save filtered list to {GENE_LIST_FINAL}")

### R Code Template for Seurat Filtering

If you're working in R with a Seurat object, use this code to filter the gene list:

```r
# Load Seurat object
library(Seurat)
seurat_obj <- readRDS("path/to/your/seurat_object.rds")

# Load PCD gene list
pcd_genes <- readLines("PCD_WOT_genes.txt")

# Get genes present in Seurat object
seurat_features <- rownames(seurat_obj)

# Filter for genes present in Seurat
pcd_genes_present <- pcd_genes[pcd_genes %in% seurat_features]

# Report
cat(sprintf("PCD genes in annotation: %d\n", length(pcd_genes)))
cat(sprintf("PCD genes in Seurat: %d (%.1f%%)\n", 
            length(pcd_genes_present), 
            100 * length(pcd_genes_present) / length(pcd_genes)))

# Write filtered list
writeLines(pcd_genes_present, "PCD_WOT_genes.present_in_seurat.txt")
```

## 11. Final Summary and WOT Usage

### Gene List Summary

**Files Created:**
1. `PCD_WOT_genes.txt` — Full list from GO annotations
2. `PCD_WOT_genes.present_in_seurat.txt` — Filtered for Seurat object (requires manual filtering)

### Using This Gene List in WOT

The filtered gene list (`PCD_WOT_genes.present_in_seurat.txt`) should be used as input to **Waddington-OT's death score calculation**.

In WOT, the death score models the gradient of cellular stress and programmed cell death. This is NOT used as a direct cell-type annotation, but rather as a **fate parameter** that influences trajectory inference.

### Biological Interpretation

This gene list captures:
- **Programmed Cell Death (PCD)**: Genetically controlled cell death processes
- **Oxidative Stress & ROS**: Reactive oxygen species signaling and homeostasis
- **Stress Responses**: Cellular and environmental stress adaptation

These processes are **plant-appropriate** and do not rely on animal apoptosis machinery.

### Publication-Ready Documentation

For methods section:

> *PCD/stress-related genes were identified from Brachypodium distachyon Bd21-3 Gene Ontology annotations (eggNOG/HuaCM) using 10 Biological Process terms: programmed cell death (GO:0012501), cell death (GO:0008219), regulation of programmed cell death (GO:0043067), regulation of cell death (GO:0010941), response to oxidative stress (GO:0006979), reactive oxygen species metabolic process (GO:0072593), cell redox homeostasis (GO:0045454), cellular response to stress (GO:0033554), response to stress (GO:0006950), and response to abiotic stimulus (GO:0009628). Protein-level GO annotations were deterministically mapped to Seurat gene IDs using the .v1.2 suffix convention. The resulting gene list was used as input to Waddington-OT death score calculation without cross-species BLAST or projection.*

In [ ]:
# Final statistics
print("=" * 80)
print("FINAL GENE LIST STATISTICS")
print("=" * 80)
print(f"\nGO Terms Used: {len(GO_TERMS)}")
print(f"  - PCD/Cell Death: 4 terms")
print(f"  - Stress/ROS: 6 terms")
print(f"\nGenes Extracted: {len(pcd_genes):,}")
print(f"  - From {len(pcd_annotations):,} GO annotations (BP only)")
print(f"  - Unique protein IDs mapped: {pcd_annotations['gene'].nunique():,}")
print(f"\nFiles Generated:")
print(f"  1. {GENE_LIST_RAW.name}")
print(f"  2. {GENE_LIST_FINAL.name} (requires Seurat filtering)")
print(f"\nNext Step:")
print(f"  → Filter gene list for genes present in your Seurat object")
print(f"  → Use filtered list as WOT death score input")
print("=" * 80)

---

## ✅ Checklist for Publication

- [x] **Native gene IDs**: Brachypodium distachyon Bd21-3 (no BLAST)
- [x] **GO-based selection**: 10 explicit Biological Process terms
- [x] **Deterministic ID mapping**: Protein (.1.p) → Gene (.v1.2)
- [x] **Plant-appropriate biology**: PCD replaces apoptosis, includes ROS/stress
- [x] **Reproducible workflow**: All GO terms documented, script versioned
- [x] **Appropriate usage**: Gene list for WOT death score, not cell annotation

### Key Message for Reviewers

> The "death score" in WOT captures a **gradient of stress and programmed cell death** specific to plants, not animal apoptosis, and is constructed entirely from native *Brachypodium distachyon* GO annotations without cross-species projections.

---

**Notebook completed successfully!**